In [ ]:
import os
import nibabel as nib
import numpy as np
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import imageio
from pathlib import Path
import shutil

DIRECTORIES = [Path('data/training/HGG'), Path('data/training/LGG')]
OUTPUT = Path('data/processed/')

In [ ]:
def rescale(im):
    # rescale image from float64 to uint8
    assert im.dtype == np.float64
    mi = np.nanmin(im)
    ma = np.nanmax(im)
    im = (im - mi) / (ma - mi) * (np.power(2.0, 8) - 1) + 0.499999999
    return im.astype(np.uint8)

In [ ]:
def save_layers(layer_id, subdir, layer_name):
    for filename in ['t1', 't1ce', 't2', 'flair']:
        scan = nib.load(str(subdir / f'{subdir.name}_{filename}.nii.gz'))
        scan = scan.get_fdata()
        # rotate scans
        scan = np.swapaxes(scan, 0, 2)
        scan_layer = scan[layer_id]
        scan_layer = rescale(scan_layer)
        imageio.imwrite(OUTPUT / subdir.name / layer_name / f'{filename}.png', scan_layer)


def save_segment(segment_layer, name, layer_name):
    ed = segment_layer == 2
    et = segment_layer == 4
    merged = np.logical_or(et, ed) 
    scaled = merged.astype(np.uint8) * 255 # make tumor white so its visible in the image file
    imageio.imwrite(OUTPUT / subdir.name / layer_name / f'segment.png', scaled)


def process(subdir):
    seg_file = subdir / f'{subdir.name}_seg.nii.gz'
    segment = nib.load(str(seg_file)).dataobj
    segment = np.swapaxes(segment, 0, 2)
    search = set(np.unique(segment))
    
    for i in range(len(segment)):
        if search == set(np.unique(segment[i])):
            start = i
            break
    for i in range(len(segment) - 1, -1, -1):
        if search == set(np.unique(segment[i])):
            end = i
            break
    length = end - start

    layer1 = start + int(length / 4)
    layer2 = start + int(length / 2)
    layer3 = start + int(length / 4 * 3)

    (OUTPUT / subdir.name).mkdir()
    
    (OUTPUT / subdir.name / 'L1').mkdir()
    (OUTPUT / subdir.name / 'L2').mkdir()
    (OUTPUT / subdir.name / 'L3').mkdir()
    
    save_segment(segment[layer1], subdir.name, 'L1')
    save_segment(segment[layer2], subdir.name, 'L2')
    save_segment(segment[layer3], subdir.name, 'L3')
    
    save_layers(layer1, subdir, 'L1')
    save_layers(layer2, subdir, 'L2')
    save_layers(layer3, subdir, 'L3')

if OUTPUT.exists():
    shutil.rmtree(OUTPUT)
OUTPUT.mkdir()

for directory in DIRECTORIES:
    for subdir in directory.iterdir():
        if not subdir.is_dir():
            continue
        process(subdir)